Christie Woodside Homework 3 - CDC Places Part 1

In [73]:
import pandas as pd
df = pd.read_csv('PLACES__Local_Data_for_Better_Health__County_Data_2024_release_20240828.csv')


/var/folders/lx/byrzd5mx73x6_9r57gkh73jh0000gp/T/ipykernel_60115/4242668162.py:2: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('PLACES__Local_Data_for_Better_Health__County_Data_2024_release_20240828.csv')


1.	investigate how many missing data points each column has.

There are 4 columns that have some missing values: Location name, data value footnote, data value footnote symbol, and Geolocation


2.	Following our class discussion, comment on whether the presence of missing data could impede the analysis of measured health characteristics. For example, consider how missing data may affect identifying patterns and significant differences at the level of locations/cities and when aggregated by states.

Data value footnote and data value footnote symbol are missing data for every single row. I think it would be interesting infromation to know, but without seeing any values I am not sure how important this information is to the analysis. 

Missing data in location name and geolocation can definitely impede the analysis. It is important to understand if there is a consistency/pattern of information coming from the same towns or area in comparison to the rest of the data.This could skew the data and display inaccurate results. For example, we could have some entries with critical measured health characteristics, but do not know what town or geolocation this information is from. We are missing information that could help us see a correlation between the data and to the location. 


In [74]:
num_columns = df.shape[1]
print(f"\nThe DataFrame has {num_columns} columns.")
# Check for missing data
m = df.isnull().sum()
print(f"\n Missing data in each column: \n{m}")


The DataFrame has 22 columns.

 Missing data in each column: 
Year                               0
StateAbbr                          0
StateDesc                          0
LocationName                      80
DataSource                         0
Category                           0
Measure                            0
Data_Value_Unit                    0
Data_Value_Type                    0
Data_Value                         0
Data_Value_Footnote_Symbol    240872
Data_Value_Footnote           240872
Low_Confidence_Limit               0
High_Confidence_Limit              0
TotalPopulation                    0
TotalPop18plus                     0
LocationID                         0
CategoryID                         0
MeasureId                          0
DataValueTypeID                    0
Short_Question_Text                0
Geolocation                       80
dtype: int64


3.	Discuss how the spatial locations are represented in the dataset (e.g., list the names of the columns related to location measures and their relationships to each other).

There is stateabbr, statedesc, locationID, locationName, and geolocation. Location name is referring to the county name and the countyId is the FIPS id for the county. The geolocation is written as its longitude and latitude. Stateabbr is the state abbreviation and statedesc lists out the full state name. 
ex: AL,Alabama,Dallas,...,01047,...POINT (-87.1065206124462 32.3258306814392)

4.	Following our class discussion, check and explain any inconsistencies and the one-to-one correspondence between ('StateAbbr' and 'StateDesc') and between/among ('LocationName', 'LocationID', and 'Geolocation').

See code below. StateAbbr and StateDesc should not (and do not) have any inconsistencies in their one-to-one correspondance. They are 1:1 and there seems to be no misspellings or anything.

LocationName will always have duplicates, because it is referring to the county. A different state could have a county by the same name. The drop_duplicates is only removing duplicates of the same abbr/state/county, btu if it is a different state then it will increase the count. If there are also any missing values this can show up as inconsistencies, which fromt he excercise above, LocationName has many missing values. If we scrubbed the data of missing values and then checked inconsistencies the number should be much lower.


In [75]:
state_a_to_d = df[['StateAbbr', 'StateDesc']].drop_duplicates()
# Check for duplicates
dup_abbr = state_a_to_d.groupby('StateAbbr').size().reset_index(name='count')
dup_desc = state_a_to_d.groupby('StateDesc').size().reset_index(name='count')

print(f'Becuase they all have the value of 1, this means there are no incorrect spellings to cause any \nduplicates when comparing one-to-one:\n{dup_abbr}')
print(f'\n{dup_desc}')
# print(dup_desc)

Becuase they all have the value of 1, this means there are no incorrect spellings to cause any 
duplicates when comparing one-to-one:
   StateAbbr  count
0         AK      1
1         AL      1
2         AR      1
3         AZ      1
4         CA      1
5         CO      1
6         CT      1
7         DC      1
8         DE      1
9         FL      1
10        GA      1
11        HI      1
12        IA      1
13        ID      1
14        IL      1
15        IN      1
16        KS      1
17        KY      1
18        LA      1
19        MA      1
20        MD      1
21        ME      1
22        MI      1
23        MN      1
24        MO      1
25        MS      1
26        MT      1
27        NC      1
28        ND      1
29        NE      1
30        NH      1
31        NJ      1
32        NM      1
33        NV      1
34        NY      1
35        OH      1
36        OK      1
37        OR      1
38        PA      1
39        RI      1
40        SC      1
41        SD      1
42    

In [76]:
state_l = df[['LocationName','LocationID', 'Geolocation']].drop_duplicates()

# Check for duplicates
dup_name = state_l.groupby('LocationName').size().reset_index(name='count')
dup_id = state_l.groupby('LocationID').size().reset_index(name='count')
dup_geo = state_l.groupby('Geolocation').size().reset_index(name='count')

#print(dup_name)
#print(dup_id)
#print(dup_geo)
dup_id = dup_id[dup_id['count'] > 1].head()
print(f'locationID\n{dup_id}\n')
dupli_name = dup_name[dup_name['count'] > 1]
print(f'location name\n {dupli_name}\n')
# print(f'location name total number of duplicates:\n{dup_name.__len__()}')
dup_geo = dup_geo[dup_geo['count'] > 1].head()
print(f'\ngeoloc\n {dup_geo}\n')


locationID
Empty DataFrame
Columns: [LocationID, count]
Index: []

location name
      LocationName  count
4           Adair      4
5           Adams     12
13         Albany      2
19      Alexander      2
25       Allegany      2
...           ...    ...
1808        Worth      3
1810       Wright      3
1813      Wyoming      3
1830         York      5
1834         Yuma      2

[442 rows x 2 columns]


geoloc
 Empty DataFrame
Columns: [Geolocation, count]
Index: []



In [77]:
if not dupli_name.empty:
    print(f'location name total number of duplicates:\n{dupli_name.__len__()}')

#redoing the df to remove the missing values focusing mainly in LocationName
df_cleaned = df.dropna()
m = df_cleaned.isnull().sum()
print(f"\n Missing data in each column of now cleaned_df: \n{m}")


location name total number of duplicates:
442

 Missing data in each column of now cleaned_df: 
Year                          0
StateAbbr                     0
StateDesc                     0
LocationName                  0
DataSource                    0
Category                      0
Measure                       0
Data_Value_Unit               0
Data_Value_Type               0
Data_Value                    0
Data_Value_Footnote_Symbol    0
Data_Value_Footnote           0
Low_Confidence_Limit          0
High_Confidence_Limit         0
TotalPopulation               0
TotalPop18plus                0
LocationID                    0
CategoryID                    0
MeasureId                     0
DataValueTypeID               0
Short_Question_Text           0
Geolocation                   0
dtype: int64


In [78]:
#recheck the inconsistencies with the cleaned df
state_l = df_cleaned[['LocationName','LocationID', 'Geolocation']].drop_duplicates()
#print(state_l)
dup_name = state_l.groupby('LocationName').size().reset_index(name='count')
dup_id = state_l.groupby('LocationID').size().reset_index(name='count')
dup_geo = state_l.groupby('Geolocation').size().reset_index(name='count')

print("there is empty dataframe, because there are no duplicates, therefore no inconsistencies,\nbecause we removed all the missing values.\n")
dup_id = dup_id[dup_id['count'] > 1].head()
print(f'locationID\n{dup_id}\n')
dupli_name = dup_name[dup_name['count'] > 1]
print(f'location name\n {dupli_name}\n')
dup_geo = dup_geo[dup_geo['count'] > 1].head()
print(f'\ngeoloc\n {dup_geo}\n')


there is empty dataframe, because there are no duplicates, therefore no inconsistencies,
because we removed all the missing values.

locationID
Empty DataFrame
Columns: [LocationID, count]
Index: []

location name
 Empty DataFrame
Columns: [LocationName, count]
Index: []


geoloc
 Empty DataFrame
Columns: [Geolocation, count]
Index: []



5.	Discuss how health outcomes are represented in the data and how many unique health outcomes are being measured.

Measure shows how they are measuring the health data, ex Stroke among adults or Hearing disability among adults. Category shows the category of the health risks like disability, health outcomes, or health status. Unique categories include 'Health Outcomes' 'Disability' 'Health Risk Behaviors' 'Health Status'
 'Prevention' and 'Health-Related Social Needs'.
 
Within these columns there is data (mainly listed as strings) on health behaviors like smoking, drinking, and physical inactivity. Mental health and physical health is also reported like depression. Disabilites are listed such as hearing loss or cognitive disability. They are mainly representedt as a description in the form of a sentence/string.

The column short_question_check also provides a single word about what health data is being measured such as STROKE, Depression, Obesity, Diabetes, etc.


In [79]:
#print(df['Measure'].head())
print(df['Measure'].dtype)

if 'Measure' in df.columns:
    uni_outcomes = df['Measure'].unique()
    print("\nnum of unique health outcomes measured:", len(uni_outcomes))
    print("\nunique health outcomes:", uni_outcomes)


object

num of unique health outcomes measured: 40

unique health outcomes: ['Diagnosed diabetes among adults' 'Hearing disability among adults'
 'Stroke among adults' 'Short sleep duration among adults'
 'Cancer (non-skin) or melanoma among adults'
 'Frequent physical distress among adults'
 'Current lack of health insurance among adults aged 18-64 years'
 'Current asthma among adults' 'Arthritis among adults'
 'Cognitive disability among adults' 'Mobility disability among adults'
 'Any disability among adults' 'Depression among adults'
 'Obesity among adults' 'Binge drinking among adults'
 'Visits to doctor for routine checkup within the past year among adults'
 'Coronary heart disease among adults'
 'Feeling socially isolated among adults'
 'High blood pressure among adults'
 'All teeth lost among adults aged >=65 years'
 'Visited dentist or dental clinic in the past year among adults'
 'Self-care disability among adults' 'Vision disability among adults'
 'Mammography use among wome

In [80]:
if 'Category' in df.columns:
    uni_cat = df['Category'].unique()
    print("\nnumber of unique health outcome categories:", len(uni_cat))
    print("\nlist of all unique categories:", uni_cat)

if 'Short_Question_Text' in df.columns:
    uni_text = df['Short_Question_Text'].unique()
    print("\nnumber of unique short q text column:", len(uni_text))
    print("\nlist of all unique texts:", uni_text)



number of unique health outcome categories: 6

list of all unique categories: ['Health Outcomes' 'Disability' 'Health Risk Behaviors' 'Health Status'
 'Prevention' 'Health-Related Social Needs']

number of unique short q text column: 40

list of all unique texts: ['Diabetes' 'Hearing Disability' 'Stroke' 'Short Sleep Duration'
 'Cancer (non-skin) or Melanoma' 'Frequent Physical Distress'
 'Health Insurance' 'Current Asthma' 'Arthritis' 'Cognitive Disability'
 'Mobility Disability' 'Any Disability' 'Depression' 'Obesity'
 'Binge Drinking' 'Annual Checkup' 'Coronary Heart Disease'
 'Social Isolation' 'High Blood Pressure' 'All Teeth Lost' 'Dental Visit'
 'Self-care Disability' 'Vision Disability' 'Mammography'
 'Independent Living Disability' 'COPD' 'High Cholesterol'
 'Cholesterol Screening' 'General Health' 'Physical Inactivity'
 'Utility Services Threat' 'Current Cigarette Smoking' 'Food Insecurity'
 'Frequent Mental Distress' 'Colorectal Cancer Screening' 'Food Stamps'
 'High Blood 

6.	Is there a time variable in the data? If so, discuss whether the repeated measures over time and the health measures of interest differ across each time period.

There is a year column referring to when the health data was collected, and from our previous homework, we configured it to be 2022 and 2021. This is not enough time to see how much the data has changed over the years, we would need more values. 

We could maybe assess the category values and see how many times a value appears in one year as opposed to another (see code below). Maybe in one year disability is never assessed or social needs. The data shows thats ome of those values are in fact missing from specific years indicating that they were looking at different health outcomes or didn't collect any data for those in different years.


In [81]:

if 'Year' in df.columns: 
    uni_years = df['Year'].unique()
    print("unique years in the dataset:", uni_years)

#to see what measures are with what years
if 'Year' in df.columns and 'Data_Value' in df.columns:
    g = df.groupby(['Year', 'Measure'])['Data_Value'].mean().reset_index() #using data value just to get the mean
    print(f'\n values of measure and its corresponding year:\n {g.head(25)}')

unique years in the dataset: [2022 2021]



 values of measure and its corresponding year:
     Year                                            Measure  Data_Value
0   2021                 Cholesterol screening among adults   82.359276
1   2021                   High blood pressure among adults   35.127550
2   2021  High cholesterol among adults who have ever be...   33.737394
3   2021  Taking medicine to control high blood pressure...   69.871475
4   2022        All teeth lost among adults aged >=65 years   15.985374
5   2022                        Any disability among adults   33.675087
6   2022                             Arthritis among adults   28.675437
7   2022                        Binge drinking among adults   17.715421
8   2022         Cancer (non-skin) or melanoma among adults    8.066534
9   2022  Chronic obstructive pulmonary disease among ad...    8.300938
10  2022                  Cognitive disability among adults   15.954674
11  2022  Colorectal cancer screening among adults aged ...   60.662623
12  2022       

In [82]:
if 'Year' in df.columns and 'Data_Value' in df.columns:
    g = df.groupby(['Year', 'Category'])['Data_Value'].mean().reset_index() #using data value just to get the mean
    print(f'\n values of category and its corresponding year:\n {g.head(9)}\n')

category_counts = df.groupby(['Year', 'Category']).size().unstack(fill_value=0)
# Display the counts
print(f"Counts of each category by year:\n{category_counts}")



 values of category and its corresponding year:
    Year                     Category  Data_Value
0  2021              Health Outcomes   34.432472
1  2021                   Prevention   76.115375
2  2022                   Disability   13.182734
3  2022              Health Outcomes   15.692806
4  2022        Health Risk Behaviors   24.755560
5  2022                Health Status   17.479348
6  2022  Health-Related Social Needs   17.326486
7  2022                   Prevention   55.527164

Counts of each category by year:
Category  Disability  Health Outcomes  Health Risk Behaviors  Health Status  \
Year                                                                          
2021               0            12312                      0              0   
2022           44030            62900                  25160          18870   

Category  Health-Related Social Needs  Prevention  
Year                                               
2021                                0       12312  
20

7.	Create a DataFrame where the rows represent unique locations and the columns represent health measures.



In [83]:
if 'LocationName' in df.columns: 
    #removing the nan locations
    Ddf= df.dropna(subset=['LocationName', 'Measure'])
    #get unique values
    unique_locations = Ddf['LocationName'].unique()
    uni_l = Ddf.groupby(['LocationName', 'Measure']).size().reset_index(name='Count')
    #pivot
    p_df = uni_l.pivot_table(index='LocationName', columns='Measure', values='Count', fill_value=0)

    p_df = p_df.reindex(index=unique_locations, fill_value=0)
    print(f'pivot table of unique locations by measure values:\n{p_df}')
    #print(uni_l)


pivot table of unique locations by measure values:
Measure       All teeth lost among adults aged >=65 years  \
LocationName                                                
Barbour                                               4.0   
Calhoun                                              22.0   
Chambers                                              4.0   
Clay                                                 36.0   
Dallas                                               10.0   
...                                                   ...   
Door                                                  2.0   
Oconto                                                2.0   
Calumet                                               2.0   
Sauk                                                  2.0   
Uinta                                                 2.0   

Measure       Any disability among adults  Arthritis among adults  \
LocationName                                                        
Barbour          

8.	List the top 3 highest correlated health measures.

The correlation coeff between the health measures of each pair is 0.996864

In [84]:
if 'LocationName' in df.columns: 
    Ddf= df.dropna(subset=['LocationName', 'Measure'])
    unique_locations = Ddf['LocationName'].unique()
    uni_l = Ddf.groupby(['LocationName', 'Measure']).size().reset_index(name='Count')
    p_df = uni_l.pivot_table(index='LocationName', columns='Measure', values='Count', fill_value=0)

    corr_matrix= p_df.corr()
    #print(corr_matrix)
    corr_matrix = corr_matrix.stack()
    corr_stack = corr_matrix[corr_matrix <1]
    top_3 = corr_stack.nlargest(3)
    print(top_3)

Measure                                      Measure                                                  
All teeth lost among adults aged >=65 years  Cholesterol screening among adults                           0.996864
                                             High blood pressure among adults                             0.996864
                                             High cholesterol among adults who have ever been screened    0.996864
dtype: float64


9.	Create a DataFrame where the rows represent states and the columns represent health measures, with the values inside the table being the averages of the measures by state.

In [113]:
if 'StateDesc' in df.columns and 'Measure' in df.columns:
    #df_clean = df.dropna(subset=['StateDesc', 'Measure']) #make sure no Nans
    #print(df.head())

    s_counts = df.groupby(['StateDesc', 'Measure']).size().reset_index(name='Count')
    #print(df[['StateDesc', 'Measure']].isna().sum())
    #print(s_counts)  # #checking to see how many
    p_df = s_counts.pivot_table(index='StateDesc', columns='Measure', values='Count', fill_value=0)
    #print(p_df)
    
    avg_df = p_df.mean(axis=0).reset_index()
    avg_df.columns = ['Measure', 'AverageCount']

    avg_row = pd.DataFrame(avg_df.set_index('Measure').T)
    avg_row.index = ['AverageCount']

    final_df = pd.concat([p_df, avg_row])
    print(final_df)
    # # merged_df = pd.merge(p_df, avg_df, left_on=p_df.columns, right_on='Measure', how='left')
    # # merged_df = merged_df.rename(columns={'AverageCount': 'AvgCount'})
    # # print(merged_df)
    


Measure               All teeth lost among adults aged >=65 years  \
Alabama                                                134.000000   
Alaska                                                  60.000000   
Arizona                                                 30.000000   
Arkansas                                               150.000000   
California                                             116.000000   
Colorado                                               128.000000   
Connecticut                                             18.000000   
Delaware                                                 6.000000   
District of Columbia                                     2.000000   
Florida                                                134.000000   
Georgia                                                318.000000   
Hawaii                                                  10.000000   
Idaho                                                   88.000000   
Illinois                          